In [28]:
import sys
import os

In [29]:
# sys.path.append()

In [30]:
sys.path.append(os.path.abspath('../../app'))
# sys.path.append(os.path.abspath('../../app/core'))


In [31]:
print(sys.path)

['/usr/local/lib/python311.zip', '/usr/local/lib/python3.11', '/usr/local/lib/python3.11/lib-dynload', '', '/usr/app/venv/lib/python3.11/site-packages', '/usr/app', '/usr/app', '/usr/app/core', '/usr/app']


In [53]:
from core.operators import vid_vec_rep_clip
from core.models.media_factory import VideoFactory
from core.store.es_vec import ES
from core.config import *
from core.models.media import MediaType
from datetime import datetime

In [33]:
# https://github.com/aatmanvaidya/audio-files/raw/main/feluda-media/en-speech.mp4
# https://github.com/aatmanvaidya/audio-files/raw/main/feluda-media/hi-speech.mp4
# https://github.com/aatmanvaidya/audio-files/raw/main/feluda-media/ta-speech.mp4

In [34]:
temp = audio_file_path = VideoFactory.make_from_url(
    r"https://github.com/aatmanvaidya/audio-files/raw/main/feluda-media/en-speech.mp4"
)

100% [..............................................................................] 48488 / 48488
Video downloaded


In [7]:
vid_vec_rep_clip.initialize({})
t = vid_vec_rep_clip.run(temp)

Installing packages for vid_vec_rep_clip


/usr/app/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/app/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [50]:
param_dict = {
    "host_name": "es",
    "text_index_name": "text",
    "image_index_name": "image",
    "video_index_name": "video",
    "audio_index_name": "audio",
}

param = QueueConfig(
    label="test",
    type="es",
    parameters=StoreESParameters(
        host_name=param_dict["host_name"],
        image_index_name=param_dict["image_index_name"],
        text_index_name=param_dict["text_index_name"],
        video_index_name=param_dict["video_index_name"],
        audio_index_name=param_dict["audio_index_name"],
    ),
)
es = ES(param)
es.connect()

In [54]:
def generate_document(post_id: str, representation: any):
    base_doc = {
        "e_kosh_id": "",
        "dataset": post_id,
        "metadata": None,
        "date_added": datetime.now().isoformat(),
    }

    def generator_doc():
        for vector in representation:
            base_doc["_index"] = "video"
            base_doc["vid_vec"] = vector["vid_vec"]
            base_doc["is_avg"] = vector["is_avg"]
            base_doc["duration"] = vector["duration"]
            base_doc["n_keyframes"] = vector["n_keyframes"]
            yield base_doc

    return generator_doc

doc = generate_document('temp.mp4',t)
media_type = MediaType.VIDEO
result = es.store(media_type, doc)
print("result:", result)


----> 6 (0, [])
result: {'message': 'multiple media stored'}
